# Ottimizzare gli iperparametri con un processo sweep

Esistono molti algoritmi di Machine Learning che richiedono iperparametri (i valori dei parametri che influenzano il training, ma non possono essere determinati dai dati di training stessi). Ad esempio, quando si esegue il training di un modello di regressione logistica, è possibile usare un iperparametro di frequenza di regolarizzazione per contrastare la distorsione nel modello; oppure quando si esegue il training di una rete neurale convoluzionale, è possibile usare iperparametri come frequenza di apprendimento e dimensioni batch per controllare la regolazione dei pesi e il numero di elementi di dati elaborati rispettivamente in un mini batch. La scelta dei valori degli iperparametri può influire in modo significativo sulle prestazioni di un modello sottoposto a training o sul tempo impiegato per eseguirne il training; e spesso è necessario provare più combinazioni per trovare la soluzione ottimale. 

## Prima di iniziare

Per eseguire il codice in questo notebook, è necessaria la versione più recente del pacchetto  **azureml-ai-ml** . Eseguire la cella seguente per verificare che sia installata.

> **Nota**:
> Se il pacchetto **azure-ai-ml** non è installato, eseguire `pip install azure-ai-ml` per installarlo.

In [ ]:
## Connettersi all'area di lavoro

Dopo aver installato i pacchetti SDK necessari, è ora possibile connettersi all'area di lavoro.

Per connettersi a un'area di lavoro, sono necessari parametri di identificatore: ID sottoscrizione, nome del gruppo di risorse e nome dell'area di lavoro. Il nome del gruppo di risorse e il nome dell'area di lavoro sono già compilati automaticamente. Per completare il comando è necessario solo l'ID sottoscrizione.

Per trovare i parametri necessari, fare clic sulla sottoscrizione e sul nome dell'area di lavoro in alto a destra di Studio. Verrà aperto un riquadro a destra.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copiare l'ID sottoscrizione e sostituire **YOUR-SUBSCRIPTION-ID** con il valore copiato. </p>

## Creare uno script di training
L'ottimizzazione degli iperparametri è ideale quando si vuole eseguire il training di modelli di Machine Learning, ma variano i parametri di input. Sarà necessario creare uno script di training che prevede un parametro di input che rappresenta uno degli iperparametri dell'algoritmo.

Eseguire le celle seguenti per creare la cartella **src** e lo script di training.

Si noti che lo script di training prevede due parametri di input:

- `--training_data` che prevede una stringa. Specificare il percorso di un asset di dati registrato come dati di training di input.
- `--reg_rate`che prevede un numero, ma ha un valore predefinito .`0.01` Questo parametro di input verrà usato per l'ottimizzazione degli iperparametri.

In [ ]:
## Configurare ed eseguire un processo di comando

Eseguire la cella seguente per eseguire il training di un modello di classificazione per stimare il diabete. Il training del modello viene eseguito eseguendo lo script **py di training\.** disponibile nella cartella **src**. Usa l'asset di dati registrato `diabetes-data` come dati di training. 

- `code`: specifica la cartella che include lo script da eseguire.
- `command`: specifica cosa eseguire esattamente.
- `environment`: specifica i pacchetti necessari da installare nel calcolo prima di eseguire il comando .
- `compute`: specifica il calcolo da usare per eseguire il comando.
- `display_name`: nome del singolo processo.
- `experiment_name`: nome dell'esperimento a cui appartiene il processo.

Si noti che il processo di comando esegue lo script di training una sola volta, con una frequenza di regolarizzazione di `0.1`. Prima di eseguire un processo di sweep per ottimizzare gli iperparametri, è consigliabile verificare se lo script funziona come previsto con un processo di comando.

In [ ]:
## Definire lo spazio di ricerca

Al termine del processo di comando, è possibile configurare ed eseguire un processo sweep. 

In primo luogo, è necessario specificare lo spazio di ricerca per l'iperparametro. Per eseguire il training di tre modelli, ognuno con una frequenza di regolarizzazione diversa (`0.01`, `0.1`o `1`), è possibile definire lo spazio di ricerca con un `Choice` iperparametro. 

## Configurare e inviare il processo sweep

Si userà la funzione sweep per eseguire l'ottimizzazione degli iperparametri nello script di training. Per configurare un processo sweep, è necessario configurare quanto segue:

- `compute`: nome della destinazione di calcolo in cui eseguire il processo.
- `sampling_algorithm`: algoritmo di campionamento degli iperparametri da usare nello spazio di ricerca. I valori consentiti sono `random`, `grid` e `bayesian`.
- `primary_metric`: nome della metrica primaria segnalata da ogni processo di valutazione. La metrica deve essere registrata nello script di training dell'utente usando `mlflow.log_metric()` con lo stesso nome della metrica corrispondente.
- `goal`: obiettivo di ottimizzazione dell'oggetto `primary_metric`. I valori consentiti sono `maximize` e `minimize`.
- `limits`: limiti per il processo sweep. Ad esempio, la quantità massima di prove o modelli di cui si vuole eseguire il training.

Si noti che il processo di comando viene usato come base per il processo sweep. La configurazione per il processo di comando verrà riutilizzata dal processo sweep.

In [ ]:
Eseguire la cella seguente per inviare il processo sweep.

In [ ]:
Al termine del processo, passare alla panoramica del processo. Nella scheda **Versioni di** valutazione verranno visualizzati tutti i modelli sottoposti a training e il modo in cui il `Accuracy` punteggio è diverso per ogni valore di frequenza di regolarizzazione provato.

## Configure and run a command job

Run the cell below to train a classification model to predict diabetes. The model is trained by running the **train\.py** script that can be found in the **src** folder. It uses the registered `diabetes-data` data asset as the training data. 

- `code`: specifies the folder that includes the script to run.
- `command`: specifies what to run exactly.
- `environment`: specifies the necessary packages to be installed on the compute before running the command.
- `compute`: specifies the compute to use to run the command.
- `display_name`: the name of the individual job.
- `experiment_name`: the name of the experiment the job belongs to.

Note that the command job only runs the training script once, with a regularization rate of `0.1`. Before you run a sweep job to tune hyperparameters, it's a best practice to test whether your script works as expected with a command job.

In [ ]:
from azure.ai.ml import command, Input
from azure.ai.ml.constants import AssetTypes

# configure job

job = command(
    code="./src",
    command="python train.py --training_data ${{inputs.diabetes_data}} --reg_rate ${{inputs.reg_rate}}",
    inputs={
        "diabetes_data": Input(
            type=AssetTypes.URI_FILE, 
            path="azureml:diabetes-data:1"
            ),
        "reg_rate": 0.01,
    },
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-mlflow",
    experiment_name="diabetes-training", 
    tags={"model_type": "LogisticRegression"}
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

## Define the search space

When your command job has completed successfully, you can configure and run a sweep job. 

First, you'll need to specify the search space for your hyperparameter. To train three models, each with a different regularization rate (`0.01`, `0.1`, or `1`), you can define the search space with a `Choice` hyperparameter. 

In [ ]:
from azure.ai.ml.sweep import Choice

command_job_for_sweep = job(
    reg_rate=Choice(values=[0.01, 0.1, 1]),
)

## Configure and submit the sweep job

You'll use the sweep function to do hyperparameter tuning on your training script. To configure a sweep job, you'll need to configure the following:

- `compute`: Name of the compute target to execute the job on.
- `sampling_algorithm`: The hyperparameter sampling algorithm to use over the search space. Allowed values are `random`, `grid` and `bayesian`.
- `primary_metric`: The name of the primary metric reported by each trial job. The metric must be logged in the user's training script using `mlflow.log_metric()` with the same corresponding metric name.
- `goal`: The optimization goal of the `primary_metric`. The allowed values are `maximize` and `minimize`.
- `limits`: Limits for the sweep job. For example, the maximum amount of trials or models you want to train.

Note that the command job is used as the base for the sweep job. The configuration for the command job will be reused by the sweep job.

In [ ]:
# apply the sweep parameter to obtain the sweep_job
sweep_job = command_job_for_sweep.sweep(
    compute="aml-cluster",
    sampling_algorithm="grid",
    primary_metric="training_accuracy_score",
    goal="Maximize",
)

# set the name of the sweep job experiment
sweep_job.experiment_name="sweep-diabetes"

# define the limits for this sweep
sweep_job.set_limits(max_total_trials=4, max_concurrent_trials=2, timeout=7200)

Run the following cell to submit the sweep job.

In [ ]:
returned_sweep_job = ml_client.create_or_update(sweep_job)
aml_url = returned_sweep_job.studio_url
print("Monitor your job at", aml_url)

When the job is completed, navigate to the job overview. The **Trials** tab will show all models that have been trained and how the `Accuracy` score differs for each regularization rate value you tried.